In [5]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import json
import re

# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop_words = (stopwords.words('english'))

In [6]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd    
poltutrain = pd.read_json(path_or_buf="poltu data/train.jsonl", lines=True)
poltuval = pd.read_json(path_or_buf="poltu data/val.jsonl", lines=True)

covidtrain = pd.read_json(path_or_buf="covid data/train.jsonl", lines=True)
covidval = pd.read_json(path_or_buf="covid data/val.jsonl", lines=True)




poltutrain = poltutrain.drop('image',axis=1)
poltuval = poltuval.drop('image',axis=1)

covidtrain = covidtrain.drop('image',axis=1)
covidval = covidval.drop('image',axis=1)

labels = {'hero': 0,
          'villain': 1,
          'victim': 2,
          'other': 3
          }



In [8]:
contractions = { 
  "ain't": "am not",
  "aren't": "are not",
  "cant":"cannot",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "bc": "because",
  "becos":"because",
  "cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "corp": "corporation",
  "cud":"could",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "execs": "executives",
  "fck": "fuck",
  "fcking": "fucking",
  "gon na": "going to",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "im": "i am",
  "iam": "i am",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "mgr": "manager",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "ofc": "office",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "pics": "pictures",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "svc":"service",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "tho":"though",
  "to've": "to have",
  "wan na": "want to",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}


def cont_to_exp(x):
    if type(x) is str:
        x = x.replace('\\','')
        for key in contractions:
            val = contractions[key]
            x = x.replace(key,val)
        return x
    return x

poltutrain['OCR'] = poltutrain['OCR'].apply(lambda x: x.lower())
poltutrain['OCR'] = poltutrain['OCR'].apply(lambda x: cont_to_exp(x))
poltutrain['OCR'] = poltutrain['OCR'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))



poltuval['OCR'] = poltuval['OCR'].apply(lambda x: x.lower())
poltuval['OCR'] = poltuval['OCR'].apply(lambda x: cont_to_exp(x))
poltuval['OCR'] = poltuval['OCR'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))



covidtrain['OCR'] = covidtrain['OCR'].apply(lambda x: x.lower())
covidtrain['OCR'] = covidtrain['OCR'].apply(lambda x: cont_to_exp(x))
covidtrain['OCR'] = covidtrain['OCR'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))



covidval['OCR'] = covidval['OCR'].apply(lambda x: x.lower())
covidval['OCR'] = covidval['OCR'].apply(lambda x: cont_to_exp(x))
covidval['OCR'] = covidval['OCR'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", x))


# traindata = traindata.sample(frac=1)


In [9]:

poltutrainlabels = []
poltutraintext = []



for i in poltutrain.index:
    sentence = poltutrain.iloc[i]['OCR']
    hero = poltutrain.iloc[i]['hero']
    victim = poltutrain.iloc[i]['victim']
    villain = poltutrain.iloc[i]['villain']
    other = poltutrain.iloc[i]['other']

    for entity in hero:
        poltutraintext.append(sentence + ' [SEP] ' + entity.lower())
        poltutrainlabels.append(labels['hero'])

    for entity in victim:
        poltutraintext.append(sentence + ' [SEP] ' + entity.lower())
        poltutrainlabels.append(labels['victim'])

    for entity in villain:

        poltutraintext.append(sentence + ' [SEP] ' + entity.lower())
        poltutrainlabels.append(labels['villain'])

    for entity in other:

        poltutraintext.append(sentence + ' [SEP] ' + entity.lower())
        poltutrainlabels.append(labels['other'])









covidtrainlabels = []
covidtraintext = []



for i in covidtrain.index:
    sentence = covidtrain.iloc[i]['OCR']
    hero = covidtrain.iloc[i]['hero']
    victim = covidtrain.iloc[i]['victim']
    villain = covidtrain.iloc[i]['villain']
    other = covidtrain.iloc[i]['other']

    for entity in hero:
        covidtraintext.append(sentence + ' [SEP] ' + entity.lower())
        covidtrainlabels.append(labels['hero'])

    for entity in victim:
        covidtraintext.append(sentence + ' [SEP] ' + entity.lower())
        covidtrainlabels.append(labels['victim'])

    for entity in villain:

        covidtraintext.append(sentence + ' [SEP] ' + entity.lower())
        covidtrainlabels.append(labels['villain'])

    for entity in other:

        covidtraintext.append(sentence + ' [SEP] ' + entity.lower())
        covidtrainlabels.append(labels['other'])



poltutraintext = [' '.join(i.split()) for i in poltutraintext]
covidtraintext = [' '.join(i.split()) for i in covidtraintext]



poltuvaltext = []
poltuvallabels = []




for i in poltuval.index:
    sentence = poltuval.iloc[i]['OCR']
    hero = poltuval.iloc[i]['hero']
    victim = poltuval.iloc[i]['victim']
    villain = poltuval.iloc[i]['villain']
    other = poltuval.iloc[i]['other']

    for entity in hero:
        poltuvaltext.append(sentence + ' [SEP] ' + entity.lower())
        poltuvallabels.append(labels['hero'])

    for entity in victim:
        poltuvaltext.append(sentence + ' [SEP] ' + entity.lower())
        poltuvallabels.append(labels['victim'])

    for entity in villain:
        poltuvaltext.append(sentence + ' [SEP] ' + entity.lower())
        poltuvallabels.append(labels['villain'])

    for entity in other:
        poltuvaltext.append(sentence + ' [SEP] ' + entity.lower())
        poltuvallabels.append(labels['other'])








covidvaltext = []
covidvallabels = []




for i in covidval.index:
    sentence = covidval.iloc[i]['OCR']
    hero = covidval.iloc[i]['hero']
    victim = covidval.iloc[i]['victim']
    villain = covidval.iloc[i]['villain']
    other = covidval.iloc[i]['other']

    for entity in hero:
        covidvaltext.append(sentence + ' [SEP] ' + entity.lower())
        covidvallabels.append(labels['hero'])

    for entity in victim:
        covidvaltext.append(sentence + ' [SEP] ' + entity.lower())
        covidvallabels.append(labels['victim'])

    for entity in villain:
        covidvaltext.append(sentence + ' [SEP] ' + entity.lower())
        covidvallabels.append(labels['villain'])

    for entity in other:
        covidvaltext.append(sentence + ' [SEP] ' + entity.lower())
        covidvallabels.append(labels['other'])


poltuvaltext = [' '.join(i.split()) for i in poltuvaltext]
covidvaltext = [' '.join(i.split()) for i in covidvaltext]


poltutraintext.extend(covidtraintext)
poltutrainlabels.extend(covidtrainlabels)

poltuvaltext.extend(covidvaltext)
poltuvallabels.extend(covidvallabels)

nettrain = [[poltutraintext[i],poltutrainlabels[i]] for i in range(len(poltutrainlabels))]
traindf = pd.DataFrame(nettrain,columns=['text','category'],dtype=int)
traindf = traindf.sample(frac=1,random_state=42)


netval = [[poltuvaltext[i],poltuvallabels[i]] for i in range(len(poltuvallabels))]
valdf = pd.DataFrame(netval,columns=['text','category'],dtype=int)



C:\Users\abhi2\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3460: FutureWarning: Could not cast to int32, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
import torch
import numpy as np
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


c:\Users\abhi2\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\abhi2\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhi2\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see acti

In [11]:
from torch import nn
from transformers import RobertaModel


class RoBertaClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(RoBertaClassifier, self).__init__()

        self.bert = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer
    

model = RoBertaClassifier()

In [12]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
LR = 1e-5
              
train(model, traindf, valdf, LR, EPOCHS)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 28/9970 [00:21<2:09:58,  1.27it/s]


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score
def fscores(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    with torch.no_grad():

        totalout = []
        totalpred = []
        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              output = output.argmax(dim=1)

              totalout.extend(test_label.cpu())
              totalpred.extend(output.cpu())
    
    print("Micro F1 score: ", f1_score(totalout, totalpred, average="micro"))
    print("Macro F1 score: ", f1_score(totalout, totalpred, average="macro"))
    print("Average F1 score: ", (f1_score(totalout, totalpred,average="micro") + f1_score(totalout, totalpred, average="macro")) / 2)
    

print("Validation Data F1 Scores")
fscores(model,valdf)

In [ ]:
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,exist_ok=True)

MODEL_NAME = "model_roberta.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(obj=model.state_dict(), f = MODEL_SAVE_PATH)